# Dimensão Tipo Manutenção

Dimensão de tipos de manutenção.

In [ ]:
spark.sql(f"""
CREATE OR REPLACE TABLE {fqtn(SCHEMA_GOLD, 'dim_tipo_manutencao')} AS
SELECT DISTINCT
    maintenance_type,
    maintenance_type as description
FROM {fqtn(SCHEMA_SILVER, 'maintenance_orders_clean')}
WHERE maintenance_type IS NOT NULL
""")